# Bulilding a fully connected neural network

## Imports

In [3]:

import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice and easy way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

## Dataset

In [4]:
batch_size = 64 # no of samples nn sees at a time

train_dataset = datasets.MNIST(
    root = 'mnist_dataset/',
    train = True, #train is set to true to download the train dataset
    transform = transforms.ToTensor(),
    download = True,
) # this is used to download the mnist dataset which is available in torchvision and the transform is used to convert it to a tensor here


train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True) # The DataLoader() returns an iterator which will generate batches of the selected batch_size as tuples of (data, labels)

test_dataset = datasets.MNIST(
    root = 'mnist_dataset/',
    train = False,
    transform = transforms.ToTensor(),
    download = True,
)

test_loader = DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle = True)
#similar to the train dataset the same ideas are used here


## Building the model

### simple NN

In [5]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__() # this calls the init function of the parent class
        self.fc1 = nn.Linear( input_size, 50) #here we initialise a fully connected hidden layer having 50 nodes
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x): # forward() is used to define the forward propogation the model is going to perform 
        x = F.relu(self.fc1(x)) #relu is used as an activation function here 
        x = self.fc2(x)
        return x

        #we have one input layer one hidden layer with 50 nodes and one output layer
        

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784 #28*28 which is the size fo the input images
num_classes = 10
learning_rate = 0.001
num_epochs = 3

In [7]:
model = NN(input_size= input_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
for epoch in range(num_epochs):
    print(f'Epoch : {epoch}')
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        #flattening it to 1 dimension i.e 28*28 to 784 ### not required for CNN

        data = data.reshape(data.shape[0], -1) # data.shape[0] is the batch_size

        #forward propogation

        scores = model(data)
        loss = criterion(scores,targets) #crossentropy loss applies softmax function so no need to apply to the output

        #zeros the previous grads to not have conflicts
        optimizer.zero_grad()

        #back_propogation
        loss.backward() #calculates gradient

        #gradient descent or adam step
        optimizer.step() #adjusts parameters to minimize loss

        

Epoch : 0
Epoch : 1
Epoch : 2


In [9]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad(): #to avoid gradient calculations as we don't need to back propogate
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0], -1) #comment out for cnn

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct)/float(num_samples)*100:.2f}"
        )
    model.train()


In [10]:
#NN
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 57666 / 60000 with accuracy 96.11
Got 9566 / 10000 with accuracy 95.66
